# E-SEIC
Selection of evolutionary instances with constraints for unbalanced datasets

In [1]:
from utils.dataset_utils import get_distribution, k_fold_cross_validation
from instance_selection.parameter.parameter import *  # 导入参数的设定
from instance_selection.operator.init_toolbox import init_toolbox_eseic
from instance_selection.operator.metrics import calculate_gmean_mauc, calculate_average_accuracy, calculate_accuracy
from instance_selection.operator.genetic_operator import selTournamentNDCD
from instance_selection.operator.ensemble import vote_result_ensembles, ensemble_individuals
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.base import clone
import scipy.io as sio  # 从.mat文件中读取数据集
import random
import warnings
import numpy as np
import os
from openpyxl import Workbook

warnings.filterwarnings("ignore")  # 忽略警告
from utils.excel_utils import save_to_excel_2


# 数据的预处理
def data_process(dataset=None, distribution=False):
    datasetname = dataset.DATASETNAME.split('.')[0]
    mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + dataset.DATASETNAME)  # 加载、划分数据集
    x = mat_data['X']
    y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y,
                                                        random_state=RANDOM_SEED)  # 划分数据集
    scaler = StandardScaler()  # 数据的标准化
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    unique_elements_all, classes_all, counts_all = get_distribution(y)  # 获取原始数据集分布
    unique_elements_train, classes_train, counts_train = get_distribution(y_train)  # 获取训练集分布
    unique_elements_test, classes_test, counts_test = get_distribution(y_test)  # 获取测试集分布
    weights_train = (1 / counts_train.astype(float)) / np.sum(1 / counts_train.astype(float))  # 计算每个类的权重，用于计算每个类别的权重
    if distribution:
        print(datasetname + f' distribution: {counts_all}')
        print(f'trainset distribution: {counts_train}')
        print(f'testset distribution: {counts_test}')
    model = MLPClassifier(hidden_layer_sizes=(dataset.HIDDEN_SIZE,), max_iter=dataset.MAX_ITER,
                          random_state=RANDOM_SEED, learning_rate_init=dataset.LEARNING_RATE)
    y_train_pred_proba = k_fold_cross_validation(model=clone(model), X=x_train, y=y_train, n_splits=N_SPLITS,
                                                 method='soft',
                                                 random_state=RANDOM_SEED)  # 交叉验证得到软标签
    # 将概率转化为预测结果
    y_train_pred = np.argmax(y_train_pred_proba, axis=1)

    Acc1, Acc2, Acc3 = calculate_accuracy(y_train_pred, y_train, weights_train)
    constraints = [Acc1, Acc2, Acc3]

    return x_train, x_test, y_train, y_test, constraints, weights_train, clone(model)


def main(x_train, y_train, model, balanced_method='random'):
    list_of_sfe = []
    ####################################种群的初始化###########################
    pop = toolbox.population(n=POPSIZE)  # 个体编码默认全为0
    pop = toolbox.init_population(pop, balanced_method=balanced_method)  # 初始化种群中的个体
    toolbox.evaluate(pop)  # 计算个体的适应度
    feasible_pop, infeasible_pop = toolbox.get_feasible_infeasible(pop)  # 得到可行解与不可行解
    # 输出可行解的数量
    print(f'第{0}代，可行解的数量为{len(feasible_pop)}')
    list_of_sfe.append(len(feasible_pop))
    ####################################种群的迭代#################################################
    for gen in range(1, NGEN + 11):
        offspring = selTournamentNDCD(pop, POPSIZE, tournsize=3)  # 锦标赛选择（1、先根据非支配排序的等级2、再根据拥挤距离）
        offspring = [toolbox.clone(ind) for ind in offspring]
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])  # 单点交叉
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]  # 二进制反转突变
            offspring[i + 1] = toolbox.mutate(offspring[i + 1], MR)[0]  # 二进制反转突变
            del offspring[i].fitness.values, offspring[i + 1].fitness.values
        #############################################################合并、去重#####################################################
        offspring = toolbox.individuals_constraints(offspring)  # 限制每个类至少有一个实例被选择
        pop = pop + offspring  # 种群的合并
        pop, _ = toolbox.remove_duplicates(pop)  # 去重
        while len(pop) < POPSIZE:  # 保证种群大小为POPSIZE
            add_individual = []
            num_add = POPSIZE - len(pop)
            for i in range(0, num_add):
                index = random.randint(0, len(offspring) - 1)  # 在0-len(offspring)范围内随机产生一个索引
                offspring[index] = toolbox.mutate(offspring[index], MR)[0]  # 选择index对应的个体进行突变
                del offspring[index].fitness.values
                add_individual.append(offspring[index])
            add_individual = toolbox.individuals_constraints(add_individual)  # 限制每个类至少有一个实例被选择
            pop = pop + add_individual  # 种群的合并
            pop, _ = toolbox.remove_duplicates(pop)  # 去重
        pop = toolbox.individuals_constraints(pop)  # 限制每个类至少有5个实例被选择
        toolbox.evaluate(pop)  # 计算新种群适应度
        ###############################################得到pareto_fronts############################################
        feasible_pop, infeasible_pop = toolbox.get_feasible_infeasible(pop)  # 得到可行解与不可行解
        # 输出可行解的数量
        print(f'第{gen}代，可行解的数量为{len(feasible_pop)}')
        list_of_sfe.append(len(feasible_pop))
        if len(feasible_pop) >= POPSIZE:
            pop, pareto_fronts = toolbox.select(feasible_pop, POPSIZE)
            ensembles = pop  # pop均为可行解，则集成pop中所有个体
        elif len(feasible_pop) > 0:
            pop = feasible_pop + infeasible_pop[:POPSIZE - len(feasible_pop)]  # 在不可行解中选取违约程度小的个体，保证pop数量为POPSIZE
            ensembles = feasible_pop  # 只集成可行解
        else:
            pop = feasible_pop + infeasible_pop[:POPSIZE - len(feasible_pop)]  # 加入不可行解中违约程度小的个体，保证pop数量为POPSIZE
            ensembles = [infeasible_pop[0]]  # 没有可行解，集成不可行解中第一个个体
        _, avg_acc2, _ = calculate_average_accuracy(ensembles)  # 计算acc1、acc2、acc3的平均值
    ensemble_classifiers = ensemble_individuals(ensembles, model, x_train, y_train)
    return ensemble_classifiers, list_of_sfe
def save_to_excel(data, save_path, filename='avg_results'):
    """
    将列表数据逐行写入Excel文件
    参数:
        data: 二维列表，每个子列表代表一行数据
        filename: 输出的Excel文件名(默认为output.xlsx)
    """
    # 创建一个新的工作簿
    wb = Workbook()
    # 获取活动的工作表
    ws = wb.active
    # 逐行写入数据
    for row in data:
        avg = row[1].tolist()
        avg.insert(0, row[0])
        std = row[2].tolist()
        std.insert(0, row[0])
        ws.append(avg)
        ws.append(std)
    # 创建 Excel 文件完整路径
    file_path = os.path.join(save_path, filename + ".xlsx")
    # 保存Excel文件
    wb.save(file_path)
    print(f"数据已成功写入到 {file_path}")

## 运行

In [2]:
# DATASETS = [Balance_Scale,Dermatology,Ecoli,Car,Pen_Digits,WallRobot,German,Wine,Nursery,Penbased,USPS,Satellite,Page_Blocks,Shuttle,Contraceptive,Automobile,Ovarian]  # 数据集名称（包含对应的参数配置）

# DATASETS = [Balance_Scale,Dermatology,Ecoli,Car,Pen_Digits,WallRobot,Wine,Satellite,Shuttle]  # 数据集名称（包含对应的参数配置）
DATASETS = [German,Nursery,Penbased,Contraceptive,USPS]  # 数据集名称（包含对应的参数配置）
#DATASETS = [Balance_Scale]  # 数据集名称（包含对应的参数配置）
if __name__ == "__main__":
    save_path = 'C:/Users/zsc/Desktop/MILE/'
    datasets_sfe_results = []
    print("*****************算法开始执行：******************")
    for j, dataset in enumerate(DATASETS):
        x_train, x_test, y_train, y_test, constraints, weights_train, model = data_process(dataset=dataset,
                                                                                           distribution=False)
        toolbox = init_toolbox_eseic(model, x_train, y_train, weights_train, constraints, n_splits=N_SPLITS,
                                     random_seed=42)  # 初始化toolbox
        num_run = 4  # 运行次数
        for i in range(num_run):
            _, list_of_sfe = main(x_train, y_train, model=model, balanced_method='random')
            datasets_sfe_results.append([dataset.DATASETNAME.split('.')[0], list_of_sfe])
    print("*****************算法执行结束！******************")
    for k, list in enumerate(datasets_sfe_results):
        print(f'{list[0]}的第{(k + 1) % (num_run+1)}次运行的结果为：{list[1]}')

*****************算法开始执行：******************
第0代，可行解的数量为7
第1代，可行解的数量为13
第2代，可行解的数量为25
第3代，可行解的数量为45
第4代，可行解的数量为47
第5代，可行解的数量为48
第6代，可行解的数量为51
第7代，可行解的数量为53
第8代，可行解的数量为55
第9代，可行解的数量为54
第10代，可行解的数量为53
第11代，可行解的数量为55
第12代，可行解的数量为53
第13代，可行解的数量为54
第14代，可行解的数量为59
第15代，可行解的数量为59
第16代，可行解的数量为59
第17代，可行解的数量为58
第18代，可行解的数量为60
第19代，可行解的数量为60
第20代，可行解的数量为60
第21代，可行解的数量为59
第22代，可行解的数量为60
第23代，可行解的数量为56
第24代，可行解的数量为59
第25代，可行解的数量为58
第26代，可行解的数量为59
第27代，可行解的数量为59
第28代，可行解的数量为59
第29代，可行解的数量为59
第30代，可行解的数量为58
第31代，可行解的数量为60
第32代，可行解的数量为57
第33代，可行解的数量为59
第34代，可行解的数量为60
第35代，可行解的数量为60
第36代，可行解的数量为58
第37代，可行解的数量为60
第38代，可行解的数量为58
第39代，可行解的数量为59
第40代，可行解的数量为59
第0代，可行解的数量为12
第1代，可行解的数量为23
第2代，可行解的数量为40
第3代，可行解的数量为48
第4代，可行解的数量为48
第5代，可行解的数量为52
第6代，可行解的数量为52
第7代，可行解的数量为56
第8代，可行解的数量为54
第9代，可行解的数量为52
第10代，可行解的数量为56
第11代，可行解的数量为54
第12代，可行解的数量为57
第13代，可行解的数量为56
第14代，可行解的数量为56
第15代，可行解的数量为57
第16代，可行解的数量为57
第17代，可行解的数量为55
第18代，可行解的数量为57
第19代，可行解的数量为56
第20代，可行解的数量为60
第21代，可行解的数量为60
第22代，可行解的数量为58
第23代，可行解的数量为58
第24

In [3]:
a=np.array(datasets_sfe_results)